# COMP5328 Assignment 2 - Label noise classification

1. [Load packages](#packages)
2. [Load datasets](#datasets)
3. [Pre-processing](#preprocessing)
4. [Experiments](#experiments)
    - 4.1 [Baseline](#baseline) 
    - 4.2 [Importance Reweighting](#rw_method1)
    - 4.3 [Reweighting method by Nagarajan Natarajan](#rw_method2)
    - 4.4 [Estimate noise rate](#estimate_rho)
5. [Reference](#Reference)
6. Packages versions
    - `python`: 3.7.0
    - `sklearn`: 0.19.2
    - `numpy`: 1.15.1
    - `matplotlib`: 2.2.3
    - `PIL`: 5.2.0
    - `skimage`: 0.14.1
    - `LIBSVM`: libsvm-weights-3.23 (Ths installation and set up can be found in README)
    
    (**Notice**: if some packages versions (e.g. skimage) are not matched, this may cause problems when run the code)


<a id="packages"></a>
## 1. Load packages 

In [1]:
# Helper libraries
import collections
import numpy as np
import numpy.linalg as lng
import time as time
from matplotlib import pyplot as plt
from PIL import Image

# Scikit-learn
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold

# LIBSVM (libsvm-weights-3.23)
# The installation and setup can be found in README
from svmutil import *

<a id="datasets"></a>
## 2. Load datasets

### NpzFile(fid)

A dictionary-like object with lazy-loading of files in the zipped
archive provided on construction.

`NpzFile` is used to load files in the NumPy ``.npz`` data archive
format. It assumes that files in the archive have a ``.npy`` extension,
other files are ignored.

The arrays and file strings are lazily loaded on either
getitem access using ``obj['key']`` or attribute lookup using
``obj.f.key``. A list of all files (without ``.npy`` extensions) can
be obtained with ``obj.files`` and the ZipFile object itself using
``obj.zip``.

In [2]:
def load_data(_):
    # This function is used to load MNIST and CIFAR dataset from NpzFile
    dataset = np.load('{}_dataset.npz'.format(_))
    Xtr = dataset['Xtr']
    Str = dataset['Str'].reshape(len(Xtr),) # Reshape 'Str' for later svm training 
    Xts = dataset['Xts']
    Yts = dataset['Yts'].reshape(len(Xts),)
    print('Load {}_dataset successfully'.format(_))
    
    return Xtr, Str, Xts, Yts

- Viewing .npy images https://stackoverflow.com/questions/33480297/viewing-npy-images

The raw data are 28 × 28 (for `Fashion-MNIST`) or 32 × 32 × 3 (for `CIFAR`) images, which are reshaped to features with dimension d = 784 or d = 3072. 

(\*Note: We just use this step to visulise the input and test if data loading is correct, we do not use any advantages from this step to train our model.)

In [3]:
dataset = np.load('mnist_dataset.npz')

x = dataset['Xtr'][0].reshape(28,28)
x.shape
img = Image.fromarray(x)
#img.show()

In [4]:
dataset = np.load('cifar_dataset.npz')

#x = dataset['Xtr'][456].reshape(1,-1).reshape(32,32,3)#.transpose()

x = dataset['Xtr'][0].reshape(3,-1).T.reshape(32,-1,3)
x.shape
img = Image.fromarray(x)
#img.show()

In [5]:
#x = dataset['Xtr'][456].reshape(1,-1).reshape(32,32,3)#.transpose()
x = dataset['Xtr'][0].reshape(3,-1).T.reshape(32,-1,3)
x.shape
img = Image.fromarray(x)
#img.show()

**Load CIFAR and MNIST dataset separately.**

In [4]:
x_train_cifar, y_train_cifar,x_test_cifar,y_test_cifar = load_data('cifar')

Load cifar_dataset successfully


In [5]:
print(x_train_cifar.shape)
print(y_train_cifar.shape)
print(x_test_cifar.shape)
print(y_test_cifar.shape)

(10000, 3072)
(10000,)
(2000, 3072)
(2000,)


In [6]:
x_train_mnist,y_train_mnist,x_test_mnist,y_test_mnist = load_data('mnist') 

Load mnist_dataset successfully


In [7]:
print(x_train_mnist.shape)
print(y_train_mnist.shape)
print(x_test_mnist.shape)
print(y_test_mnist.shape)

(10000, 784)
(10000,)
(2000, 784)
(2000,)


<a id="preprocessing"></a>
## 3. Pre-processing

In [10]:
#x_train_mnist_reduce = skimage.measure.block_reduce(x_train_mnist, (1,2), np.max)
#x_test_mnist_reduce = skimage.measure.block_reduce(x_test_mnist, (1,2), np.max)
#print(x_train_mnist_reduce.shape)
#print(x_test_mnist_reduce.shape)

In [8]:
def norm(x):
    # min-max normalisation
    return (x-x.min())/(x.max()-x.min()) 

def cal_pca(train,test,threshold = 0.95):
    '''
    input:
    train - train data
    test - test data
    threshold - percentage of variance kept after pca
    '''
    pca = PCA(threshold)
    train -= train.mean()
    pca.fit(train)
    train_ = pca.transform(train)
    test -= test.mean()
    test_ = pca.transform(test)
    
    return train_,test_

In [9]:
# Normalise and dimension reduction of MNIST dataset
xtr_mnist,xts_mnist = cal_pca(norm(x_train_mnist),norm(x_test_mnist),0.95)
#xtr_mnist,xts_mnist = cal_pca(x_train_mnist,x_test_mnist,0.95)
str_mnist,yts_mnist = y_train_mnist,y_test_mnist

# Normalise and dimension reduction of CIFAR datset
xtr_cifar,xts_cifar = cal_pca(norm(x_train_cifar),norm(x_test_cifar),0.95)
#xtr_cifar,xts_cifar = cal_pca(x_train_cifar,x_test_cifar,0.95)
str_cifar,yts_cifar = y_train_cifar,y_test_cifar

# Print dimensions 
print('MNIST dataset train set feature dimensions:', xtr_mnist.shape)
print('MNIST dataset test set feature dimensions:', xts_mnist.shape)
print('MNIST dataset train set label dimensions:', str_mnist.shape)
print('MNIST dataset test set label dimensions:', yts_mnist.shape)
print('\nCIFAR dataset train set feature dimensions:', xtr_cifar.shape)
print('CIFAR dataset test set feature dimensions:', xts_cifar.shape)
print('CIFAR dataset test set label dimensions:', str_cifar.shape)
print('CIFAR dataset test set label dimensions:', yts_cifar.shape)

MNIST dataset train set feature dimensions: (10000, 129)
MNIST dataset test set feature dimensions: (2000, 129)
MNIST dataset train set label dimensions: (10000,)
MNIST dataset test set label dimensions: (2000,)

CIFAR dataset train set feature dimensions: (10000, 205)
CIFAR dataset test set feature dimensions: (2000, 205)
CIFAR dataset test set label dimensions: (10000,)
CIFAR dataset test set label dimensions: (2000,)


<a id="experiments"></a>
## 4. Experiments

We use libsvm-weights-3.23 to train our model (https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/weights/libsvm-weights-3.23.zip)

**Instrunction of how to use libsvm**: 

### `svm-train` set params
```
Usage: svm-train [options] training_set_file [model_file]
options:
-s svm_type : set type of SVM (default 0)
        0 -- C-SVC            (multi-class classification)
        1 -- nu-SVC           (multi-class classification)
        2 -- one-class SVM
        3 -- epsilon-SVR      (regression)
        4 -- nu-SVR           (regression)
-t kernel_type : set type of kernel function (default 2)
        0 -- linear: u'*v
        1 -- polynomial: (gamma*u'*v + coef0)^degree
        2 -- radial basis function: exp(-gamma*|u-v|^2)
        3 -- sigmoid: tanh(gamma*u'*v + coef0)
        4 -- precomputed kernel (kernel values in training_set_file)
-c cost : set the parameter C of C-SVC, epsilon-SVR, and nu-SVR (default 1)
-b probability_estimates : whether to train a SVC or SVR model for probability estimates, 0 or 1 (default 0)
-v n: n-fold cross validation mode
-q : quiet mode (no outputs)
```
More details: https://github.com/cjlin1/libsvm/blob/master/README#L134

In [16]:
def evaluate_prediction(y, X, model, options='-b 1 -q'):
    """
    y: a list/tuple/ndarray of labels
    X: a list/tuple of training instances
    model: trained SVM model
    
    p_labels: a list of predicted labels
    p_acc: a tuple including  accuracy (for classification), mean-squared
           error, and squared correlation coefficient (for regression).
    p_vals: a list of decision values or probability estimates (if '-b 1'
            is specified). If k is the number of classes, for decision values,
            each element includes results of predicting k(k-1)/2 binary-class
            SVMs. For probabilities, each element contains k values indicating
            the probability that the testing instance is in each class.
            Note that the order of classes here is the same as 'model.label'
            field in the model structure.
    """
    
    p_labels, p_acc, p_vals = svm_predict(y, X, model, options)
    ACC, MSE, SCC = evaluations(y, p_labels)
    return ACC

<a id="baseline"></a>
### 4.1. Baseline

In [14]:
%%time
# Set W = [] if no weights are given
# Train a base svm and get baseline test accuracy for Fashion-MNIST dataset
model_mnist = svm_train([], str_mnist, xtr_mnist.tolist(), '-b 1')
ACC_mnist = evaluate_prediction(yts_mnist, xts_mnist, model_mnist)

print('Accuracy of test set of original svm model trained on Fashion-MNIST: %.3f' %ACC_mnist)

Accuracy of test set of original svm model trained on Fashion-MNIST: 86.900
CPU times: user 2min 24s, sys: 2.89 s, total: 2min 27s
Wall time: 2min 38s


In [15]:
%%time
# Train a base svm and get baseline test accuracy for CIFAR dataset
model_cifar = svm_train([], str_cifar, xtr_cifar.tolist(), '-b 1')
ACC_cifar = evaluate_prediction(yts_cifar, xts_cifar, model_cifar)

print('Accuracy of test set of original svm model trained on CIFAR: %.3f' %ACC_cifar)

Accuracy of test set of original svm model trained on CIFAR: 73.350
CPU times: user 3min 46s, sys: 4.01 s, total: 3min 50s
Wall time: 4min 8s


<a id="rw_method1"></a>
### 4.2. Importance Reweighting

In [17]:
# rho_0 and rho_1 are given 
rho0 = 0.2
rho1 = 0.4

\begin{equation}
\beta(x,y) = \frac{p_{\rho}(S=y|X=x)-\rho_{1-y}}{(1-\rho_{0}-\rho_{1})p_{\rho}(S=y|X=x)},
\end{equation}
    where $p_{\rho}(X,Y)$ can be estimated from noisy data.

In [18]:
def estimateBeta(S, prob, rho0, rho1):
    '''
    *Adapted from tutorial 11
    input:
    S - Str, label with noise
    prob - probablity returned from trained svm
    rho0 - given value, which is 0.2
    rho1 - given value, which is 0.4
    
    return:
    beta - 1d array, weight
    '''
    beta = np.zeros((len(S),1)) # Initialise beta
    for i in range(len(S)):    
        if S[i] == 1:
            beta[i] = (prob[i][1]-rho0)/((1-rho0-rho1)*prob[i][1]+1e-5)
        else:
            beta[i] = (prob[i][0]-rho1)/((1-rho0-rho1)*prob[i][0]+1e-5)
            
    # remove negative weights
    # below is a simple condition to ensure convexity of weight beta
    for j in range(len(beta)):
        if beta[j] < 0:
            beta[j] = 0.0

    return beta

In [25]:
def evaluate_model_1(dataset_xtr, dataset_str, dataset_xts, dataset_yts, loop=10):
    '''
    Evaluation for "Importance Reweighting" model
    input:
    dataset_xtr: training set features
    dataset_str: training set labels
    dataset_xts: test set features
    dataset_yts: test set labels 
    '''    
    
    ACC_test_total = []
    
    for i in range(loop):
        # independently and randomly sample 8000 examples from train set for training model
        X_train, X_test, y_train, y_test = \
        train_test_split(dataset_xtr, dataset_str, \
                         test_size=0.2, random_state=0, shuffle=True)
        
        train_data, vali_data = X_train.tolist(), X_test.tolist()
        train_class, vali_class = y_train, y_test
        
        # test data
        test_data = dataset_xts.tolist()
      
        # Train SVM model
        model_train = svm_train([],train_class, train_data,'-b 1')
        
        # Compute probability, accuracy 
        p_label, p_acc, probS = svm_predict(train_class, train_data, model_train, '-b 1 -q')

        # ****************** Importance reweighting *****************************
        
        weights = estimateBeta(train_class, probS, rho0, rho1) # Estimate beta and compute beta
        
        # Apply beta to the loss and train the model again 
        clf_svm = svm_train(weights, train_class, train_data, '-b 1')
        ACC_test = evaluate_prediction(dataset_yts, test_data, clf_svm)

        print('Loop %d, Acc on test set: %.3f' % (i+1, ACC_test))     
        # Append accuracy result on test
        ACC_test_total.append(ACC_test)

    # Print evaluation results with average accuracy and std on test data
    print('\nPerformance evaluation for "Importance Reweighting": %.3f +/- %.3f' % (np.mean(ACC_test_total),
                                                                                    np.std(ACC_test_total)))    

In [24]:
%%time
evaluate_model_1(xtr_mnist, str_mnist, xts_mnist, yts_mnist)

Loop 1, Acc on test set: 93.550
Loop 2, Acc on test set: 93.500
Loop 3, Acc on test set: 93.500
Loop 4, Acc on test set: 93.500
Loop 5, Acc on test set: 93.500
Loop 6, Acc on test set: 93.550
Loop 7, Acc on test set: 93.500
Loop 8, Acc on test set: 93.550
Loop 9, Acc on test set: 93.500
Loop 10, Acc on test set: 93.500

Performance evaluation for "Importance Reweighting": 93.515 +/- 0.023
CPU times: user 23min 43s, sys: 21.6 s, total: 24min 5s
Wall time: 25min 39s


In [26]:
%%time
evaluate_model_1(xtr_cifar, str_cifar, xts_cifar, yts_cifar)

Loop 1, Acc on test set: 80.200
Loop 2, Acc on test set: 80.200
Loop 3, Acc on test set: 80.150
Loop 4, Acc on test set: 79.950
Loop 5, Acc on test set: 79.950
Loop 6, Acc on test set: 79.850
Loop 7, Acc on test set: 80.350
Loop 8, Acc on test set: 79.950
Loop 9, Acc on test set: 80.300
Loop 10, Acc on test set: 80.350

Performance evaluation for "Importance Reweighting": 80.125 +/- 0.176
CPU times: user 46min 40s, sys: 45.4 s, total: 47min 26s
Wall time: 50min 42s


<a id="rw_method2"></a>
### 4.3. Reweighting method by Nagarajan Natarajan
(https://www.cs.cmu.edu/~pradeepr/paperz/learning_nl_nips.pdf)

In [27]:
def sort_data(x, y):
    # seperate label of 0 and 1
    z = list(zip(x,y))
    # sort label, used for later weights multiplication  
    x_,y_ = zip(*sorted(z,key=lambda x:x[1])) 
    return x_,y_

In [35]:
def evaluate_model_2(dataset_xtr, dataset_str, dataset_xts, dataset_yts, loop=10):
    # Create empty list
    ACC_test_total = []
    
    for i in range(loop):
        
        # independently and randomly sample 8000 examples from train set for training model
        X_train, X_test, y_train, y_test = \
        train_test_split(dataset_xtr, dataset_str, test_size=0.2, \
                         random_state=0, shuffle=True)
        
        train_data, train_class = sort_data(X_train, y_train)
        vali_data, vali_class = sort_data(X_test,y_test)
        
        # test data
        test_data = dataset_xts.tolist()
        
        # Get the number of label == 0 and number of label == 1    
        num_0 = collections.Counter(train_class)[0]
        num_1 = collections.Counter(train_class)[1]
        # Construct weight list
        #alpha = (0.5-rho0)/(1-rho0-rho1)
        alpha = (1-rho1+rho0)/2
        A = list(np.ones(num_0)*alpha) + list(np.ones(num_1)*(1-alpha))        
#-------------------------------------------------------------------------------------     
        # Train SVM model
        model_train = svm_train(A, train_class, train_data, '-b 1')
        ACC_test = evaluate_prediction(dataset_yts, test_data, model_train)
        
        print('Fold: % d, Acc on test set: %.3f' % (i+1, ACC_test))          
        # Append data
        ACC_test_total.append(ACC_test)
    # Print 5-cv results with mean and std
    print('\n5-fold CV accuracy: %.3f +/- %.3f' % (np.mean(ACC_test_total), 
                                                  np.std(ACC_test_total)))    

In [36]:
%%time
evaluate_model_2(xtr_mnist, str_mnist, xts_mnist, yts_mnist)

Fold:  1, Acc on test set: 90.750
Fold:  2, Acc on test set: 90.700
Fold:  3, Acc on test set: 90.700
Fold:  4, Acc on test set: 90.700
Fold:  5, Acc on test set: 90.700
Fold:  6, Acc on test set: 90.700
Fold:  7, Acc on test set: 90.700
Fold:  8, Acc on test set: 90.700
Fold:  9, Acc on test set: 90.750
Fold:  10, Acc on test set: 90.700

5-fold CV accuracy: 90.710 +/- 0.020
CPU times: user 14min 48s, sys: 9.13 s, total: 14min 57s
Wall time: 15min 32s


In [37]:
%%time
evaluate_model_2(xtr_cifar, str_cifar, xts_cifar, yts_cifar)

Fold:  1, Acc on test set: 77.300
Fold:  2, Acc on test set: 77.350
Fold:  3, Acc on test set: 77.300
Fold:  4, Acc on test set: 77.400
Fold:  5, Acc on test set: 77.000
Fold:  6, Acc on test set: 77.050
Fold:  7, Acc on test set: 76.900
Fold:  8, Acc on test set: 77.150
Fold:  9, Acc on test set: 77.300
Fold:  10, Acc on test set: 76.800

5-fold CV accuracy: 77.155 +/- 0.197
CPU times: user 22min 45s, sys: 13.8 s, total: 22min 59s
Wall time: 24min 15s


<a id="estimate_rho"></a>
### 4.4. Estimate noise rate

In [25]:
def estimate_noise_rate(prob, y_noise):
    prob_1 = [prob[i][0] for i in range(len(prob)) if y_noise[i] == 0]
    prob_0 = [prob[i][1] for i in range(len(prob)) if y_noise[i] == 1]

    top_10_idx_1 = np.argsort(np.array(prob_1))[:10]
    top_10_values_1 = [prob_1[i] for i in top_10_idx_1]
    
    top_10_idx_0 = np.argsort(np.array(prob_0))[:10]
    top_10_values_0 = [prob_0[i] for i in top_10_idx_0]
    
    return np.mean(top_10_values_1), np.mean(top_10_values_0)

In [26]:
def estimate_rho(X, y, loop=5):
    """
    input:
    X - array-like, shape = (n_samples, n_features)
    y - array-like, shape = (n_samples) or (n_samples, n_outputs)
    loop- int, looping times [Optional]
    return:
    estimated rho0 - scalar
    estimated rho1 - scalar
    """
    
    estimated_rho0 = []
    estimated_rho1 = []

    for i in range(loop):
        print('\nLoop %d' %(i+1))
        X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.2, random_state=42)

        train_data = X_train.tolist() # change array type to list
        train_label = y_train
        W = np.ones(len(train_label))
        %%time
        # Train C-SVC SVM model
        model_train = svm_train(W, train_label, train_data, '-b 1 -s 0 -q')
        p_label, p_acc, p_val = svm_predict(train_label, train_data, model_train, '-b 1')
        rho1,rho0 = estimate_noise_rate(p_val,y_train)
        estimated_rho0.append(rho0)
        estimated_rho1.append(rho1)

    print('\nestimated noise rate of rho1 is %.3f +/- %.3f' %(np.mean(estimated_rho1),
                                                            np.std(estimated_rho1)))
    print('estimated noise rate of rho0 is %.3f +/- %.3f' %(np.mean(estimated_rho0),
                                                            np.std(estimated_rho0)))
    

**Estimate noise rate on Fashion-MNIST Dataset**

In [27]:
%%time

estimate_rho(xtr_mnist, str_mnist, loop=5)


Loop 1
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs
Accuracy = 69.4625% (5557/8000) (classification)

Loop 2
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs
Accuracy = 69.475% (5558/8000) (classification)

Loop 3
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs
Accuracy = 69.45% (5556/8000) (classification)

Loop 4
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs
Accuracy = 69.45% (5556/8000) (classification)

Loop 5
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs
Accuracy = 69.45% (5556/8000) (classification)

estimated noise rate of rho1 is 0.266 +/- 0.001
estimated noise rate of rho0 is 0.160 +/- 0.001
CPU times: user 6min 18s, sys: 1.31 s, total: 6min 19s
Wall time: 6min 20s


**Estimate noise rate on CIFAR Dataset**

In [28]:
%%time

estimate_rho(xtr_cifar, str_cifar, loop=5)


Loop 1
CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs
Accuracy = 76.7375% (6139/8000) (classification)

Loop 2
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
Accuracy = 77.0375% (6163/8000) (classification)

Loop 3
CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs
Accuracy = 76.7% (6136/8000) (classification)

Loop 4
CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.91 µs
Accuracy = 76.9875% (6159/8000) (classification)

Loop 5
CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.96 µs
Accuracy = 76.8% (6144/8000) (classification)

estimated noise rate of rho1 is 0.251 +/- 0.005
estimated noise rate of rho0 is 0.199 +/- 0.003
CPU times: user 9min 42s, sys: 1.58 s, total: 9min 44s
Wall time: 9min 45s


### Reference

- https://www.python.org/dev/peps/pep-0008/
- https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/weights/libsvm-weights-3.23.zip

Method 3

In [ ]:
# rho_0 and rho_1 are given 
rho0 = 0.2
rho1 = 0.4

In [ ]:
def estimateBeta(S, prob, rho0, rho1):
    '''
    *Adapted from tutorial 11
    input:
    S - Str, label with noise
    prob - probablity returned from trained svm
    rho0 - given value, which is 0.2
    rho1 - given value, which is 0.4
    
    return:
    alpha - 1d array, weight
    '''
    alpha = np.zeros((len(S),1)) # Initialise beta
    for i in range(len(S)):    
        if S[i] == 1:
            alpha[i] = (prob[i][1](rho0+rho1)-rho0)/((1-rho0-rho1)+1e-5)
        else:
            alpha[i] = (prob[i][0](rho0+rho1)-rho1)/((1-rho0-rho1)+1e-5)
            
    # remove negative weights
    # below is a simple condition to ensure convexity of weight beta
    for j in range(len(alpha)):
        if alpha[j] < 0:
            alpha[j] = 0.0

    return alpha

In [25]:
def evaluate_model_3(dataset_xtr, dataset_str, dataset_xts, dataset_yts, loop=10):
    '''
    Evaluation for "Importance Reweighting" model
    input:
    dataset_xtr: training set features
    dataset_str: training set labels
    dataset_xts: test set features
    dataset_yts: test set labels 
    '''    
    
    ACC_test_total = []
    
    for i in range(loop):
        # independently and randomly sample 8000 examples from train set for training model
        X_train, X_test, y_train, y_test = \
        train_test_split(dataset_xtr, dataset_str, \
                         test_size=0.2, random_state=0, shuffle=True)
        
        train_data, vali_data = X_train.tolist(), X_test.tolist()
        train_class, vali_class = y_train, y_test
        
        # test data
        test_data = dataset_xts.tolist()
      
        # Train SVM model
        model_train = svm_train([],train_class, train_data,'-b 1')
        
        # Compute probability, accuracy 
        p_label, p_acc, probS = svm_predict(train_class, train_data, model_train, '-b 1 -q')

        # ****************** Importance reweighting *****************************
        
        weights = estimateBeta(train_class, probS, rho0, rho1) # Estimate beta and compute beta
        
        # Apply beta to the loss and train the model again 
        clf_svm = svm_train(weights, train_class, train_data, '-b 1')
        ACC_test = evaluate_prediction(dataset_yts, test_data, clf_svm)

        print('Loop %d, Acc on test set: %.3f' % (i+1, ACC_test))     
        # Append accuracy result on test
        ACC_test_total.append(ACC_test)

    # Print evaluation results with average accuracy and std on test data
    print('\nPerformance evaluation for "Importance Reweighting": %.3f +/- %.3f' % (np.mean(ACC_test_total),
                                                                                    np.std(ACC_test_total)))    

In [ ]:
%%time
evaluate_model_3(xtr_mnist, str_mnist, xts_mnist, yts_mnist)

In [ ]:
%%time
evaluate_model_3(xtr_cifar, str_cifar, xts_cifar, yts_cifar)